# Running SPARQL queries

First we'll load the medium size KG serialized as `tmp.ttl` in the previous example:

In [1]:
import kglab

namespaces = {
    "wtm": "http://purl.org/heals/food/",
    "ind": "http://purl.org/heals/ingredient/",
    }

kg = kglab.KnowledgeGraph(
    name = "A recipe KG example based on Food.com",
    base_uri = "https://www.food.com/recipe/",
    language = "en",
    namespaces = namespaces,
    )

kg.load_ttl("tmp.ttl")

Now let's run a query based on [SPARQL 1.1](https://www.w3.org/TR/sparql11-query/) then iterate through the result set.
Let's look for recipes that use *eggs* and require at least 1.5 hours to prepare.

In [2]:
query = """
        SELECT ?a ?def ?time
           WHERE {
              ?a rdf:type wtm:Recipe .
              ?a skos:definition ?def .
              ?a wtm:hasIngredient ind:ChickenEgg .
              ?a wtm:hasCookTime ?time .
              FILTER (?time >= 90)
           }
           ORDER BY DESC(?time)
        """

for row in kg.query(query):
    url, recipe_name, cook_time = row
    print("{}\n {}\n {} minutes\n".format(url, recipe_name, cook_time))

https://www.food.com/recipe/31041
 marti s homemade noodles
 160 minutes

https://www.food.com/recipe/124106
 hungarian noodles homemade
 120 minutes

https://www.food.com/recipe/371915
 crepes    alton brown
 100 minutes

https://www.food.com/recipe/77354
 wonton wrappers
 90 minutes

https://www.food.com/recipe/497918
 easy homemade noodles
 90 minutes

https://www.food.com/recipe/19104
 all purpose dinner crepes batter
 90 minutes



That's interesting ... notice the outliers based on cooking times -- i.e., recipes for special kinds of *noodles*. Some of these take 2-3 hours to prepare.

## Using SPARQL queries and post-processing to create annotations

Let's begin to add [*annotations*](https://www.w3.org/TR/annotation-model/) to the KG.
In other words, we'll add descriptions to the recipes to help classify them based on a taxonomy.

We've created a brief *controlled vocabulary* in RDF called `NOM` in the `nom.ttl` file, which provides part of a simple taxonomy for our recipes.
Take a look at those definitions – mostly these are distinctions between two kind of flour-based recipes:

  * "pancakes" -- which use batter in their preparation
  * "noodles" -- which use kneading in their preparation
  
Let's load those definitions into our KG:

In [5]:
kg.load_ttl("nom.ttl")

In [3]:
query = """
        SELECT ?a ?def
           WHERE {
              ?a rdf:type wtm:Recipe .
              ?a skos:definition ?def .
              ?a wtm:hasIngredient ind:ChickenEgg .
              ?a wtm:hasIngredient ind:AllPurposeFlour .
              ?a wtm:hasIngredient ind:Salt
              }
        """

for row in kg.query(query):
    url, recipe_name = row
    has_butter = (url, kg.get_ns("wtm").hasIngredient, kg.get_ns("ind").Butter) in kg._g
    sez_noodle =  any([x in recipe_name for x in ["noodle", "spaetzle", "dumpling"]])
    print("{}\n {}\n {} {}\n".format(url, recipe_name, has_butter, sez_noodle))

https://www.food.com/recipe/458
 basic crepes ii
 True False

https://www.food.com/recipe/331765
 homemade spaetzle
 False True

https://www.food.com/recipe/148900
 homemade pasta  fettuccine
 False False

https://www.food.com/recipe/48178
 oven pancake puff
 True False

https://www.food.com/recipe/137357
 dane dumplings
 True True

https://www.food.com/recipe/96087
 fresh made manicotti or cannelloni crepes shells
 False False

https://www.food.com/recipe/72129
 muffin tin popovers
 False False

https://www.food.com/recipe/508734
 slovak egg dumplings
 False True

https://www.food.com/recipe/252783
 croatian flour noodles
 True True

https://www.food.com/recipe/136111
 small batch popovers
 False False

https://www.food.com/recipe/368491
 unleavened bread
 False False

https://www.food.com/recipe/268209
 honey wheat popovers
 True False

https://www.food.com/recipe/105830
 no beat popovers
 False False

https://www.food.com/recipe/109196
 galuska for stew  dumplings
 True True

https:

> Can we build a PSL model for *link prediction* based on whether a recipe that uses Eggs, Salt, Flour, and no butter is a kind of "noodle" ?